In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
#import seaborn as sns
import pymongo
from pprint import pprint

In [3]:
client = pymongo.MongoClient()
db = client.metrics

In [4]:
def group_by(df, bycols, agg_map):
    """

    @param df:      DataFrame
    @param bycols:  str or list
                        Column(s) to group by
    @param agg_map: dictionary or list of 2-tuples
                        Mapping from column to aggregate function e.g. [("city", "count"), ("salary", "mean"]
    @return:        DataFrame
                        Flattened dataframe, with multi-level index removed
    """
    grps = []
    if type(bycols) == str:
        bycols = [bycols]

    if type(agg_map) == dict:
        agg_map = agg_map.items()

    for k,v in agg_map:
        grp = df[bycols + [k]].groupby(bycols, ).agg(v)
        grp.reset_index(inplace=True)
        grp["%s(%s)" % (v,k)] = grp[k]
        del grp[k]
        grps.append(grp)

    m = grps[0]
    for grp in grps[1:]:
        m = pd.merge(m, grp, on=bycols, how="inner")
    return m

In [5]:
from bson.son import SON # needed to ensure dictionary is ordered (python default is not)
import hashlib

def hash_feats(fts):
    vals = fts.values
    joined = "|".join(map(lambda s: str(s),vals)).encode('utf-8') 
    return hashlib.sha224(joined).hexdigest()

def get_df_sorted_by_f1score(collection, params=None, filter_cols=True):
    if not params:
        params = []
    if type(params) == str:
        params = params.split(",")
    
    project = {
            "weighted_f1_score":"$WEIGHTED_MEAN_CONCEPT_CODES.f1_score",
            "micro_f1_score":  "$MICRO_F1.f1_score",
            "micro_recall":    "$MICRO_F1.recall",
            "micro_precision": "$MICRO_F1.precision",
    
    # PARAMETERS            
            "window_size":    "$parameters.window_size",
            "feats":          "$parameters.extractors",
            "count": {        "$size" : "$parameters.extractors" },
            "asof" :          "$asof",
            "_id":1
    }
    
    # No count for HMM
    if "_hmm" in collection.lower():
        del project["count"]
    
    for param in params:
        project[param] = "$parameters." + param

    feats_pipeline = [{
        "$project": project
    },
    {
        "$match":{
            "micro_f1_score": { "$exists" : True }        
        }
    },
    {
        "$sort":{
            "micro_f1_score": -1
        }
    },
    ]
    
    rows = [row for row in db[collection].aggregate(feats_pipeline)]
    df = pd.DataFrame(rows).sort_values("micro_f1_score", ascending=False)
    if params:
        df["hs_params"] = df[params].apply(hash_feats, axis=1)
        
    if filter_cols:
        cols = ["micro_f1_score", "micro_recall" ,"micro_precision" ] + params
        return df[cols]
    return df

In [70]:
from Metrics import rpf1a_from_tp_fp_tn_fn
from collections import defaultdict

def tally_counts(r, filter):
    tally = defaultdict(int)
    for k,v in r.items():
        if filter(k):
            for prop in "tp,tn,fp,fn".split(","):
                tally[prop] += v[prop]
    return tally

def get_causal_relation_metrics(collection, params):
    dicts = []
    for r in db[collection].find({}):
        d = {}
        cr_counts = tally_counts(r, lambda c: "->" in c)
        (rec, p, cr_f1, a) = rpf1a_from_tp_fp_tn_fn(cr_counts["tp"],cr_counts["fp"],cr_counts["tn"],cr_counts["fn"])
        d["cr_micro_f1"] = cr_f1
        d["cr_micro_rec"]  = rec
        d["cr_micro_prec"] = p
        concept_counts = tally_counts(r, lambda c: c[0].isdigit())
        (rec, p, concept_f1, a) = rpf1a_from_tp_fp_tn_fn(concept_counts["tp"],concept_counts["fp"],concept_counts["tn"],concept_counts["fn"])
        d["concept_micro_f1"] = concept_f1
        d["concept_micro_rec"]  = rec
        d["concept_micro_prec"] = p
        parms = r["parameters"]
        for p in params:
            d[p] = parms[p]
        dicts.append(d)
    df = pd.DataFrame(dicts)
    fields = ("cr_micro_f1,cr_micro_rec,cr_micro_prec,concept_micro_f1,concept_micro_rec,concept_micro_prec," + ",".join(params)).split(",")
    return df[fields].sort_values("cr_micro_f1", ascending=False)

In [71]:
def round_data(df, places=3):
    df_copy = df.copy()
    fmt_str = "{0:." + str(places) + "f}"
    cols = set([v for v in df_copy.columns.values if "micro_" in v])
    for c in cols:
        df_copy[c] = df[c].apply(lambda d: fmt_str.format(d))  
    return df_copy

## Coral Bleaching

In [76]:
params = "merge_mode,num_rnns,use_pretrained_embedding,bi-directional,hidden_size"
df = get_causal_relation_metrics("CR_CB_TAGGING_TD_RNN", params.split(","))
df

,cr_micro_f1,cr_micro_rec,cr_micro_prec,concept_micro_f1,concept_micro_rec,concept_micro_prec,merge_mode,num_rnns,use_pretrained_embedding,bi-directional,hidden_size
0,0.518782,0.879989,0.367809,0.654618,0.859068,0.528775,sum,2,True,True,256
5,0.506784,0.873677,0.356905,0.676103,0.879382,0.549159,sum,2,True,True,128
4,0.483229,0.871821,0.334247,0.680893,0.868731,0.559844,sum,2,True,True,64
3,0.456040,0.902450,0.305112,0.657186,0.882521,0.523516,sum,1,True,True,256
1,0.419745,0.861054,0.277514,0.663246,0.902624,0.524221,sum,1,True,True,64
2,0.415723,0.861890,0.273923,0.659545,0.887388,0.524800,sum,1,True,True,128


In [82]:
params = "merge_mode,num_rnns,use_pretrained_embedding,bi-directional,hidden_size"
df = get_causal_relation_metrics("CR_CB_TAGGING_VD_RNN", params.split(","))
df

KeyError: "['cr_micro_f1' 'cr_micro_rec' 'cr_micro_prec' 'concept_micro_f1'\n 'concept_micro_rec' 'concept_micro_prec' 'merge_mode' 'num_rnns'\n 'use_pretrained_embedding' 'bi-directional' 'hidden_size'] not in index"

In [80]:
params = "merge_mode,num_rnns,use_pretrained_embedding,bi-directional,hidden_size"
df = get_causal_relation_metrics("CR_SC_TAGGING_VD_RNN", params.split(","))
df

,cr_micro_f1,cr_micro_rec,cr_micro_prec,concept_micro_f1,concept_micro_rec,concept_micro_prec,merge_mode,num_rnns,use_pretrained_embedding,bi-directional,hidden_size
3,0.484352,0.843185,0.339760,0.642076,0.720491,0.579055,sum,2,True,True,64
5,0.455363,0.830117,0.313730,0.650665,0.770004,0.563354,sum,2,True,True,256
4,0.454093,0.834741,0.311876,0.645350,0.756189,0.562850,sum,2,True,True,128
2,0.423979,0.853840,0.282005,0.652526,0.783709,0.558963,sum,1,True,True,256
1,0.412054,0.857660,0.271167,0.652635,0.795093,0.553470,sum,1,True,True,128
0,0.392536,0.837354,0.256355,0.657309,0.789235,0.563170,sum,1,True,True,64
